In [1]:
import ee
import geemap

Using web map services

In [2]:
Map = geemap.Map()

esa_wms = 'https://services.terrascope.be/wms/v2' # The WMS URL
tcc_layer = 'WORLDCOVER_2020_S2_TCC' # The true color composite imagery
fcc_layer = 'WORLDCOVER_2020_S2_FCC' # The false color composite imagery
map_layer = 'WORLDCOVER_2020_MAP'    # The land cover classification map

Map.add_wms_layer(esa_wms, layers=tcc_layer, name='True Color', attribution='ESA')
Map.add_wms_layer(esa_wms, layers=fcc_layer, name='False Color', attribution='ESA')
Map.add_wms_layer(esa_wms, layers=map_layer, name='Classification', attribution='ESA')

Map.add_legend(title='ESA Land Cover', builtin_legend='ESA_WorldCover')
Map


Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Using Earth Engine

In [3]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

esa = ee.ImageCollection("ESA/WorldCover/v100").first()
esa_vis = {'bands': ['Map']}

Map.addLayer(esa, esa_vis, "ESA Land Cover")
Map.add_legend(title="ESA Land Cover", builtin_legend='ESA_WorldCover')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Creating charts

In [5]:
histogram = geemap.image_histogram(esa, scale=1000, x_label='Land Cover Type', y_label='Area (km2)')
histogram

In [6]:
df = geemap.image_histogram(esa, scale=1000, return_df=True)
df

,key,value
0,10,6.139810e+07
1,20,1.211905e+07
2,30,4.470917e+07
3,40,1.665006e+07
4,50,8.475490e+05
5,60,2.858194e+07
6,70,8.461224e+06
7,80,3.219519e+07
8,90,4.268191e+06
9,95,1.941690e+05


In [7]:
esa_labels = list(geemap.builtin_legends['ESA_WorldCover'].keys())
esa_labels

['10 Trees',
 '20 Shrubland',
 '30 Grassland',
 '40 Cropland',
 '50 Built-up',
 '60 Barren / sparse vegetation',
 '70 Snow and ice',
 '80 Open water',
 '90 Herbaceous wetland',
 '95 Mangroves',
 '100 Moss and lichen']

In [8]:
df['label'] = esa_labels
df

,key,value,label
0,10,6.139810e+07,10 Trees
1,20,1.211905e+07,20 Shrubland
2,30,4.470917e+07,30 Grassland
3,40,1.665006e+07,40 Cropland
4,50,8.475490e+05,50 Built-up
5,60,2.858194e+07,60 Barren / sparse vegetation
6,70,8.461224e+06,70 Snow and ice
7,80,3.219519e+07,80 Open water
8,90,4.268191e+06,90 Herbaceous wetland
9,95,1.941690e+05,95 Mangroves


In [9]:
round(df['value'].sum() / 1e6, 2)

217.06

In [10]:
geemap.bar_chart(df, x='label', y='value', x_label='Land Cover Type', y_label='Area (km2)')

In [11]:
geemap.pie_chart(df, names='label', values='value', height=500)


Adding Administrative Boundaries

In [12]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path('countries'))
africa = countries.filter(ee.Filter.eq('CONTINENT', 'Africa'))
style = {'fillColor': '00000000'}
Map.addLayer(countries.style(**style), {}, 'Countries', False)
Map.addLayer(africa.style(**style), {}, 'Africa')
Map.centerObject(africa)
Map

Map(bottom=454.0, center=[19.973348786110613, 0], controls=(WidgetControl(options=['position', 'transparent_bg…

Extracting Croplands¶

In [13]:
cropland = esa.eq(40).clipToCollection(africa).selfMask()
Map.addLayer(cropland, {'palette': ['f096ff']}, 'Cropland')
Map.show_layer(name='ESA Land Cover', show=False)

AttributeError: 'Map' object has no attribute 'show_layer'

Zonal Statistics

In [14]:
geemap.zonal_stats(cropland, africa, 'esa_cropland.csv',statistics_type='SUM', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /home/yoviajo/Documentos/lab/geom/90/esa_cropland.csv


In [15]:
df = geemap.csv_to_df('esa_cropland.csv')
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,5235.411765,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,40286.113725,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,279.831373,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,175513.933333,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,8584.458824,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia


In [16]:
geemap.bar_chart(df, x='NAME', y='sum', max_rows=30, x_label='Country', y_label='Area (km2)')

In [17]:
geemap.pie_chart(df, names='NAME', values='sum', max_rows=20, height=500)